In [708]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from collections import Counter
pd.options.display.max_rows = 4000
from sklearn.model_selection import train_test_split
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [695]:
movies = pd.read_csv('/Users/jacobtryba/DSI/assignments/capstone2/data/imdb-extensive-dataset/IMDb movies.csv')
movies_subset = ['reviews_from_critics','genre','description', 'metascore','director','imdb_title_id', 'title', 'year','duration', 'country','budget', 'usa_gross_income', 'worlwide_gross_income']
movies_subbed = movies[movies_subset]
movies_current_usa = movies_subbed.query('country == "USA"')
movies_current_usa_nonnull_ugi = movies_current_usa[(movies_current_usa.usa_gross_income.notnull())]
movies_current_usa_nonnull_ugi_budget = movies_current_usa_nonnull_ugi[(movies_current_usa_nonnull_ugi.budget.notnull())]
final_set = movies_current_usa_nonnull_ugi_budget.sort_values('year', ascending = True)
final_set['budget'] = final_set['budget'].str.replace('$', '')
final_set['budget'] = final_set['budget'].str.replace('$ ', '')
final_set['budget'] = final_set['budget'].str.replace('GBP ', '')
final_set['budget'] = final_set['budget'].str.replace('AUD ', '')
final_set['budget'] = final_set['budget'].str.replace('EUR ', '')
final_set['budget'] = final_set['budget'].str.replace('ESP ', '')
final_set['budget'] = final_set['budget'].str.replace('CAD ', '')
final_set['usa_gross_income'] = final_set['usa_gross_income'].str.replace('$ ', '')
final_set['usa_gross_income'] = final_set['usa_gross_income'].str.replace('$', '').astype('int')
final_set['worlwide_gross_income_gross_income'] = final_set['worlwide_gross_income'].str.replace('$ ', '')
final_set['worlwide_gross_income'] = final_set['worlwide_gross_income'].str.replace('$', '').astype('int')
final_set['international_gross_income'] = (final_set['worlwide_gross_income'] - final_set['usa_gross_income'])
final_set['returns'] = (final_set['worlwide_gross_income'] - final_set['budget'].astype('int'))
final_set['profitable'] = [1 if x > 0 else 0 for x in final_set['returns']]
# final_set = final_set[(final_set.metascore.notnull())]

In [696]:
# Title
final_set['title'] = final_set['title'].astype(str)
final_set['title'] = final_set['title'].str.lower()


# Description
final_set['description'] = final_set['description'].astype(str)
final_set['description'] = final_set['description'].str.lower()

# Director
final_set['director'] = final_set['director'].astype(str)
final_set['director'] = final_set['director'].str.lower()

#Genre
final_set['genre'] = final_set['genre'].astype(str)
final_set['genre'] = final_set['genre'].str.lower()

In [697]:
punct_to_remove = string.punctuation

In [698]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','', punct_to_remove))

In [699]:
final_set['description'] = final_set['description'].apply(lambda text: remove_punctuation(text))
final_set['title'] = final_set['title'].apply(lambda text: remove_punctuation(text))
final_set['genre'] = final_set['genre'].apply(lambda text: remove_punctuation(text))

In [700]:
stop_words = set(stopwords.words('english'))

In [701]:
def remove_stopwords(text):
    return ' '.join([word for word in str(text).split() if word not in stop_words])

In [702]:
# Remove Stop Words From Description of Movie + genre

final_set['description'] = final_set['description'].apply(lambda text: remove_stopwords(text))
final_set['genre'] = final_set['genre'].apply(lambda text: remove_stopwords(text))

In [ ]:
# Split My Data

# X_train, X_test, y_train, y_test = 
# train_test_split(X, y, test_size=0.33, random_state=42)

msk = np.random.rand(len(final_set)) < .8

In [ ]:
train = final_set[msk]
test = final_set[~msk]

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
len(final_set)

In [660]:
train, test = train_test_split(final_set, test_size =0.2, random_state=143278497)

In [661]:
train_copy = train.copy()

In [662]:
test_copy = test.copy()

In [663]:
train_copy['text'] = train_copy['title'] + ' ' + train_copy['director'] + ' ' + train_copy['genre']

In [664]:
test_copy['text'] = test_copy['title']  + ' ' + test_copy['director'] + ' ' + test_copy['genre']

In [665]:
two_col = ['title', 'director', 'text','profitable']

In [666]:
all_train_two_col = train_copy[two_col]
all_test_two_col = test_copy[two_col]

In [450]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = vectorizer.fit_transform(all_train_two_col['text'].values)
classifier = MultinomialNB()
targets = all_train_two_col['profitable'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [303]:
test = all_test_two_col['text']
example_counts = vectorizer.transform(test)
predictions = classifier.predict(example_counts)
predictions_movies = pd.DataFrame((all_test_two_col),columns=['movies', 'profitable', 'predictions'] )

In [304]:
tested_data = all_test_two_col.copy() 

In [305]:
tested_data['prediction'] = predictions

In [306]:
tested_data['correct'] = np.where(tested_data['prediction'] == tested_data['profitable'], 1, 0)

In [307]:
len(tested_data)

890

In [308]:
accuracy = tested_data.correct.sum() / tested_data.correct.count()

In [309]:
print(accuracy)

0.6067415730337079


In [310]:
tested_data['profitable'].sum()

533

In [311]:
tested_data['profitable'].count()

890

In [312]:
525/(890)

0.5898876404494382

In [531]:
all_train_two_col.head(1)

,title,director,text,profitable
25488,stone cold,craig r. baxley,stone cold craig r. baxley action crime drama,0


In [533]:
all_test_two_col.head(1)

,title,director,text,profitable
9993,psycho,alfred hitchcock,psycho alfred hitchcock horror mystery thriller,1


In [458]:
########## SPLIT AND TRAIN AND TEST ON TITLE, DIRECTOR, GENRE

In [532]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = vectorizer.fit_transform(all_train_two_col['text'].values)
classifier = MultinomialNB()
targets = all_train_two_col['profitable'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [534]:
test = all_test_two_col['text']
example_counts = vectorizer.transform(test)
predictions = classifier.predict(example_counts)


In [535]:
tested_data = all_test_two_col.copy() 

In [536]:
tested_data['prediction'] = predictions

In [537]:
tested_data['correct'] = np.where(tested_data['prediction'] == tested_data['profitable'], 1, 0)

In [542]:
tested_data.head(10)

,title,director,text,profitable,prediction,correct
9993,psycho,alfred hitchcock,psycho alfred hitchcock horror mystery thriller,1,1,1
34846,monsters inc,"pete docter, david silverman","monsters inc pete docter, david silverman anim...",1,1,1
22554,ishtar,elaine may,ishtar elaine may action adventure comedy,0,1,0
66478,ride along 2,tim story,ride along 2 tim story action comedy crime,1,1,1
23790,driving miss daisy,bruce beresford,driving miss daisy bruce beresford drama,1,0,0
27583,the return of the texas chainsaw massacre,kim henkel,the return of the texas chainsaw massacre kim ...,0,1,0
49520,super troopers 2,jay chandrasekhar,super troopers 2 jay chandrasekhar comedy crim...,1,1,1
22113,ruthless people,"jim abrahams, david zucker","ruthless people jim abrahams, david zucker com...",1,1,1
13118,planet of the apes,franklin j. schaffner,planet of the apes franklin j. schaffner adven...,1,1,1
21355,mask,peter bogdanovich,mask peter bogdanovich biography drama,1,0,0


In [545]:
true_positives = np.where((tested_data['prediction'] == 1) & (tested_data['profitable'] == 1), 1, 0)

In [547]:
true_negatives = np.where((tested_data['prediction'] == 0) & (tested_data['profitable'] == 0), 1, 0)

In [553]:
false_positives = np.where((tested_data['prediction'] == 1) & (tested_data['profitable'] == 0), 1, 0)

In [554]:
false_negatives = np.where((tested_data['prediction'] == 0) & (tested_data['profitable'] == 1), 1, 0)

In [675]:
acc = (true_positives.sum()+true_negatives.sum())/tested_data.count()

In [556]:
precision = true_positives.sum()/(true_positives.sum()+false_positives.sum())

In [558]:
recall = true_positives.sum()/(true_positives.sum()+false_negatives.sum())

In [674]:
acc

title         0.605618
director      0.605618
text          0.605618
profitable    0.605618
prediction    0.605618
correct       0.605618
dtype: float64

In [725]:
precision

0.6196769456681351

In [715]:
recall

0.8210116731517509

In [667]:
from scipy.sparse import csr_matrix

In [668]:
review = np.zeros((final_set['title'].count(), 3))

In [669]:
review[:,0] = np.where(final_set['metascore'] <= 40,1,0)
review[:,1] = np.where(final_set['metascore'] <= 70 ,1,0) - review[:,0]
review[:,2] = np.where(final_set['metascore'] <= 100 ,1,0) - review[:,0] - review[:,1]

In [672]:
review

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [671]:
final_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3540 entries, 1005 to 81059
Data columns (total 17 columns):
reviews_from_critics                  3540 non-null float64
genre                                 3540 non-null object
description                           3540 non-null object
metascore                             3540 non-null float64
director                              3540 non-null object
imdb_title_id                         3540 non-null object
title                                 3540 non-null object
year                                  3540 non-null int64
duration                              3540 non-null int64
country                               3540 non-null object
budget                                3540 non-null object
usa_gross_income                      3540 non-null int64
worlwide_gross_income                 3540 non-null int64
worlwide_gross_income_gross_income    3540 non-null object
international_gross_income            3540 non-null int64
returns

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0


In [710]:
final_set = final_set.sort_values(by = 'title')

In [713]:
final_set.query('director == "billy bob thornton"')

,reviews_from_critics,genre,description,metascore,director,imdb_title_id,title,year,duration,country,budget,usa_gross_income,worlwide_gross_income,worlwide_gross_income_gross_income,international_gross_income,returns,profitable
32239,87.0,drama romance western,two texas cowboys head mexico search work soon...,55.0,billy bob thornton,tt0149624,all the pretty horses,2000,116,USA,57000000,15540353,18133495,$ 18133495,2593142,-38866505,0
29321,105.0,drama,karl childers simple man hospitalized since ch...,84.0,billy bob thornton,tt0117666,sling blade,1996,135,USA,1000000,24444121,24444121,$ 24444121,0,23444121,1
